In [44]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import numpy as np
# import StringIO
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout, Merge
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
np.random.seed(7)

In [2]:
%run ../../../prepare_data.py

# import sys
# sys.path.insert(0, '../../')
# import preprare_data

In [22]:
additional_data = [el[2] for el in content]

possible_variants = sorted(set(additional_data))

In [9]:
np.array(X_other_features).shape

(536431, 167)

In [40]:
      
X_other_features = create_X_features(content, feature_dictionary)

In [12]:
feature_dictionary = create_feature_dictionary(content)
feature_dictionary

[[25,
  'A',
  ['f', 'o', 's'],
  ['c', 'p', 's'],
  ['f', 'm', 'n'],
  ['d', 'p', 's'],
  ['a', 'd', 'g', 'i', 'l', 'n'],
  ['-', 'n', 'y'],
  ['-'],
  ['n', 'y']],
 [5, 'C', ['c', 's'], ['c', 's']],
 [1, 'I'],
 [25,
  'M',
  ['c', 'm', 'o', 's'],
  ['f', 'm', 'n'],
  ['d', 'p', 's'],
  ['a', 'd', 'g', 'i', 'l', 'n'],
  ['l'],
  ['-', 'n', 'y'],
  ['-'],
  ['-'],
  ['n', 'y']],
 [18,
  'N',
  ['c'],
  ['f', 'm', 'n'],
  ['d', 'p', 's'],
  ['a', 'd', 'g', 'i', 'l', 'n'],
  ['-'],
  ['-'],
  ['n', 'y']],
 [48,
  'P',
  ['d', 'g', 'i', 'p', 'q', 'r', 's', 'x', 'z'],
  ['-', '1', '2', '3'],
  ['-', 'f', 'm', 'n'],
  ['-', 'd', 'p', 's'],
  ['-', 'a', 'd', 'g', 'i', 'l', 'n'],
  ['-', 'd', 'p', 's'],
  ['-', 'f', 'm', 'n'],
  ['-', 'n', 'y'],
  ['-', 'p', 's'],
  ['a', 'n'],
  ['-'],
  ['n', 'y']],
 [1, 'Q'],
 [5, 'R', ['g'], ['c', 'p', 's']],
 [9, 'S', ['p'], ['c', 's'], ['a', 'd', 'g', 'i', 'l']],
 [30,
  'V',
  ['c', 'm', 'o'],
  ['c', 'i', 'm', 'n', 'p', 'u'],
  ['-', 'f', 'p', 's'],
 

In [5]:
# X_train, X_other_features_train, y_train, X_validate, X_other_features_validate, y_validate = generate_full_matrix_inputs()
# save_inputs('../../internal_representations/inputs/shuffeled_matrix_train_inputs_other_features.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffeled_matrix_validate_inputs_other_features.h5', X_validate, y_validate,  other_features = X_other_features_validate)
X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffeled_matrix_train_inputs_other_features.h5', other_features=True)
X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffeled_matrix_validate_inputs_other_features.h5', other_features=True)

In [24]:
# X_other_features = create_X_features(content)
# print (X_other_features[178200])
decode_position = 30
print (decode_input(X_train[decode_position], dictionary))
# print (X_other_features_train[0])
print (decode_X_features(feature_dictionary, [X_other_features_train[decode_position]]))
# print (len(X_other_features_train[0]))

navleèeta
Vmip3d--n
None


In [29]:
# X = X[:100000]
# y = y[:100000]
# def unison_shuffled_copies(a, b):
#     assert len(a) == len(b)
#     p = np.random.permutation(len(a))
#     return a[p], b[p]
# X, y = unison_shuffled_copies(X, y)
# print X.shape

# train_X = X.astype(np.float32)
# train_y = y.astype(np.int32)

num_examples = len(X_train) # training set size
# nn_input_dim = max_word * len(dictionary) # input layer dimensionality
# nn_output_dim = max_num_vowels * max_num_vowels # output layer dimensionality
nn_output_dim = 121
nn_hdim = 516

# Gradient descent parameters (I picked these by hand)
# epsilon = 1 # learning rate for gradient descent
# reg_lambda = 1 # regularization strength

In [11]:
nn_output_dim

121

In [46]:
word_processor = Sequential()
word_processor.add(Conv1D(43, (3), input_shape=(23, 43), padding='same', activation='relu'))
word_processor.add(Conv1D(43, (3), padding='same', activation='relu'))
word_processor.add(MaxPooling1D(pool_size=2))
word_processor.add(Flatten())
word_processor.add(Dense(516, activation='relu', kernel_constraint=maxnorm(3)))

metadata_processor = Sequential()
metadata_processor.add(Dense(256, input_dim=167, activation='relu'))

model = Sequential()
model.add(Merge([word_processor, metadata_processor], mode='concat'))  # Merge is your sensor fusion buddy
model.add(Dense(516, input_dim=(516 + 256), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(nn_output_dim, activation='sigmoid'))


# np.random.seed(7)

# # create model
# model = Sequential()
# model.add(Conv1D(43, (3), input_shape=(23, 43), padding='same', activation='relu'))
# model.add(Dropout(0.2))
# model.add(Conv1D(43, (3), padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# # model.add(Conv1D(43, (3), input_shape=(None, 43), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Flatten())
# # model.add(Dense(1032, activation='relu', kernel_constraint=maxnorm(3)))
# # model.add(Dropout(0.2))
# model.add(Dense(516, activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Dropout(0.2))
# model.add(Dense(nn_output_dim, activation='sigmoid'))

/home/luka/miniconda3/lib/python3.5/site-packages/ipykernel_launcher.py:12: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if sys.path[0] == '':


In [47]:
epochs = 5
lrate = 0.1
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Compile model
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [50]:
# model.fit_generator((X, y), 536432/2, epochs=5)
# testX = X[:536432/16]
# print testX.shape
# testY = y[:536432/16]
# print testY.shape
model.fit([X_train, X_other_features_train], y_train, validation_data=([X_validate, X_other_features_validate], y_validate), epochs=10, batch_size=10)
# model.fit(X, y, epochs=5, validation_split=0.2, batch_size=10)

Train on 427470 samples, validate on 108961 samples
Epoch 1/10
  1230/427470 [..............................] - ETA: 1790s - loss: 0.0012 - acc: 0.9081

KeyboardInterrupt: 

In [14]:
model.save('cnn_i2_s_c43-3relu_d20_c43-3relu_mp2_f_516relu_d20_121sigmoid_mse_adam_a65_3epoch_no_overfitting.h5')

In [10]:
model = load_model('../internal_representations/models/cnn_i2_s_c43-3relu_d20_c43-3relu_mp2_f_516relu_d20_121sigmoid_mse_adam_a65_10epoch.h5')

In [6]:
h5f.close()

NameError: name 'h5f' is not defined

In [12]:
# evaluate the model
scores = model.evaluate(X[429145:], y[429145:])
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

107232/107287 [============================>.] - ETA: 0s
acc: 96.52%


In [118]:
# calculate predictions
# test = generate_input_from_word('test', max_word, dictionary)
# print test.shape
# print test.T.shape
predictions = model.predict(np.array([generate_input_from_word('biotski', max_word, dictionary)]))
# round predictions

print decode_position(predictions[0], max_num_vowels)

b_pred = 0
ind = 0
i = 0
for el in predictions[0]:
    if b_pred < el:
        b_pred = el
        ind = i
    i += 1
    
print(ind)
# rounded = [round(x[0]) for x in predictions]
# print(rounded)

[2, 0]
2


In [ ]:
predictions = model.predict(X[429145:])

In [21]:
decode_position(predictions[0], max_num_vowels)

[1, 0]

In [23]:
def test_accuracy(predictions, y):
    dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
    num_of_pred = len(predictions)
    num_of_correct_pred = 0
    for i in range(predictions.shape[0]):
        if decode_position(predictions[i], max_num_vowels) == decode_position(y[i], max_num_vowels):
            num_of_correct_pred += 1

    return (num_of_correct_pred/float(num_of_pred)) * 100


print(test_accuracy(predictions, y[429145:]))

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
107287
96.5186835311


In [17]:
predictions.shape

(107287, 121)

In [12]:
print max_num_vowels

11


In [8]:
dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
def generate_input_from_word(word, max_word, dictionary):
    x = np.zeros((max_word, len(dictionary)))
    j = 0
    for c in list(word):
        index = 0
        for d in dictionary:
            if c == d:
                x[j, index] = 1
                break
            index += 1
        j += 1
    return x

# model = load_model()
# prediction = predict(model, generate_input_from_word('hidrija'))
# print decode_position(prediction[0])

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
